# Raster Transformations
- this process is responsible for applying the profile curvature and residual topography transformations to the extent dems.
- Because the profile curvature step still needs to be executed in QGIS, this step requires user input
- Because there is no vector processing yet, and therefore no requisite matching of vector to raster data, this script can easily be run as a batch process on a folder of tifs



In [ ]:
from pathlib import Path

import numpy as np
import rasterio
import matplotlib.pyplot as plt

from scrollstats.delineation import CalcResidualTopography, profile_curvature_instructions
from globals import RASTER_DIRS, WINDOW_SIZE

# Define Inputs


## Create `dem_paths`
- gather and sort all of the dems into a list
- This process expects the `input/dem` folder to contain the DEMs that cover the extent of all bends of interest to the user
                                                                                           


In [ ]:
## regex-like pattern matching to filter dems; 
## leave as "*" to match all files in `dem_dir`
regex_str = "sb*.tif"

## function used to sort the dems within `dem_dir`
## simply return `x` to use default filename sorting
def sort_func(x):
    return x.stem.split("_")[2]


dem_paths = sorted(RASTER_DIRS["dem"].glob(regex_str), key = sort_func)
dem_paths[:5]

## Calculate Residual Topography
- Loop over `dem_paths` and calculate the residual topography transform for each dem present
- Save all residual topography rasters to `output/rt`

In [ ]:
for dem_path in dem_paths[:3]:
    rt = CalcResidualTopography(dem_path, WINDOW_SIZE, RASTER_DIRS["rt"])
    rt_path = rt.execute()
    print(f"Calculated rt transform for {dem_path.name}")

## DEM vs Residual Topography example 

In [ ]:
dem = rasterio.open(dem_path).read(1)
dem[dem<-1e6] = np.nan

rt = rasterio.open(rt_path).read(1)

fig, (ax1,ax2) = plt.subplots(1, 2, figsize=(20, 15))
mapper = ax1.imshow(dem)
# fig.colorbar(mapper, ax=ax1)
ax1.set_axis_off()
ax1.set_title("Original DEM")


mapper = ax2.imshow(rt)
fig.colorbar(mapper, ax=ax2)
ax2.set_axis_off()
ax2.set_title("Residual Topography");


## Instructions for QGIS implementation of profile curvature

Profile curvature is calculated using the `r.param.scale` tool in QGIS3

In [ ]:
profile_curvature_instructions()